<a href="https://colab.research.google.com/github/justcme/PerformanceSpecs/blob/main/B_Dictionaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lipid Panel Performance Recommendations
### Create Dictionary Tables

*   Encoded variable dictionary.
*   Old Allowable error and BV lookup table.
*   New Allowable error and BV lookup table.
*   PCE coefficients lookup table.

# SETUP

OS

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/Colab Notebooks/LipidPerf")

Mounted at /content/drive/


Imports

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt

# DATA

In [ ]:
PCE_coeffs = pd.read_csv("PCE_coeffs.csv")   #Coefficients for Pooled Cohort Equation calculation

#Dictionaries

Encoded Variables

In [ ]:
codebookD = {'Variable':['sex', 'sex', 'race', 'race', 'smoker', 'smoker', 'diabetes', 'diabetes'],
            'Category': ['male', 'female', 'other', 'black', 'non-smoker', 'smoker', 'non-diabetic', 'diabetic'],
            'Code': [0, 1]*4}
codebook = pd.DataFrame(codebookD)
codebook

,Variable,Category,Code
0,sex,male,0
1,sex,female,1
2,race,other,0
3,race,black,1
4,smoker,non-smoker,0
5,smoker,smoker,1
6,diabetes,non-diabetic,0
7,diabetes,diabetic,1


Old Error Data

*   HDL >=42 mg/dL: CV = 0.04% 
*   HDL <42 mg/dL: SD = 1.7mg/dL

In [ ]:
#DataFrame containing NCEP allowable error (1995) and BV from the EFLM BV Database (as in 2022)
TAE = {"Parameter": ["TC","HDL","TG","dLDL"],"BV": [0.053, 0.058, 0.2, 0.083], "PB": [0.03, 0.05, 0.05, 0.04], "CV": [0.03, 0.04, 0.05, 0.04], "SD": [np.nan, 1.7, np.nan, np.nan]}
TAE_df = pd.DataFrame(TAE)
TAE_df

,Parameter,BV,PB,CV,SD
0,TC,0.053,0.03,0.03,NaN
1,HDL,0.058,0.05,0.04,1.7
2,TG,0.200,0.05,0.05,NaN
3,dLDL,0.083,0.04,0.04,NaN


New Error Data

*   new PB: HDL: 4%, TC: 2%, TG: 4%  
*   new CV: 3% for all

**   maintained minimum SD of 1.7%, adjusted the threshold for CV vs SD

***   HDL >=57 mg/dL: CV = 0.03% 

***   HDL <57 mg/dL: SD = 1.7mg/dL

In [ ]:
#DataFrame containing proposed new allowable error and BV from the EFLM BV Database (as in 2022)
TAEnew = {"Parameter": ["TC","HDL","TG","dLDL"],"BV": [0.053, 0.058, 0.2, 0.083], "PB": [0.02, 0.04, 0.04, 0.04], "CV": [0.03, 0.03, 0.03, 0.04], "SD": [np.nan, 1.7, np.nan, np.nan]}
TAEnew_df = pd.DataFrame(TAEnew)
TAEnew_df

,Parameter,BV,PB,CV,SD
0,TC,0.053,0.02,0.03,NaN
1,HDL,0.058,0.04,0.03,1.7
2,TG,0.200,0.04,0.03,NaN
3,dLDL,0.083,0.04,0.04,NaN


PCE Coefficients Column Name

In [ ]:
PCE_coeffs.rename(columns= {'Unnamed: 0':'group'}, inplace = True)   # Renamed an un-named column
glist = ['black male', 'black female', 'other male', 'other female']
for i,g in zip(range(4), glist):
  PCE_coeffs.loc[i,'group'] = g

Argument 'PB' for analytical and sample replicate analysis

In [ ]:
PBd = {'Scenario': ['None', 'Positive Bias', 'Negative Bias'], 'PB':[0,1,2]}
PB = pd.DataFrame(PBd)

# Save new dataframes to .CSV files in *Base* folder

In [ ]:
codebook.to_csv('from_Pandas/Base/codebook.csv', index = False)
TAE_df.to_csv('from_Pandas/Base/TAE_df.csv', index = False)
TAEnew_df.to_csv('from_Pandas/Base/TAEnew_df.csv', index = False)
PCE_coeffs.to_csv('from_Pandas/Base/PCE_coeffs.csv', index = False)
PB.to_csv('from_Pandas/Base/PB.csv', index = False)